<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Papers/blob/main/P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

tf.__version__

'2.11.0'

In [ ]:
vgg16 = tf.keras.applications.vgg16.VGG16(weights='imagenet')

553467096/553467096 [==============================] - 4s 0us/step


In [ ]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
for i in range(len(vgg16.layers)):
  vgg16.layers[i].trainable=False
  print(vgg16.layers[i].name)

input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool
flatten
fc1
fc2
predictions


In [ ]:
def ftb(current_layer, former_layer=None):
  x = tf.keras.layers.Conv2D(filters=256, kernel=3, stride=1, activation='relu', padding='same')(current_layer)
  x = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same')(x)

  if former_layer != None:
    d = tf.keras.layers.Conv2DTranspose(filters=256, kernel_size=4, strides=2, padding='same')(former_layer)
    x = tf.keras.layers.Add()([x, d])

  return x

In [23]:
def rfem(x):
  channel = x.shape[-1]
  #branch1
  print(x.shape)
  x1 = tf.keras.layers.MaxPool2D(pool_size=2, padding='same', strides=1)(x)
  print(x1.shape)
  x1 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=1, padding='same', activation='relu')(x1)
  print(x1.shape)
  x1 = tf.keras.layers.BatchNormalization()(x1)
  print(x1.shape)

  #branch2
  x2 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=1, padding='same', activation='relu')(x)
  x2 = tf.keras.layers.BatchNormalization()(x2)
  x2 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(1, 3), padding='same', activation='relu')(x2)
  x2 = tf.keras.layers.BatchNormalization()(x2)

  #branch3
  x3 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=1, padding='same', activation='relu')(x)
  x3 = tf.keras.layers.BatchNormalization()(x3)
  x3 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(3, 1), padding='same', activation='relu')(x3)
  x3 = tf.keras.layers.BatchNormalization()(x3)

  #branch4
  x4 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=1, padding='same', activation='relu')(x)
  x4 = tf.keras.layers.BatchNormalization()(x4)
  x4 = tf.keras.layers.Conv2D(filters=channel//3, kernel_size=(3, 1), padding='same', activation='relu')(x4)
  x4 = tf.keras.layers.BatchNormalization()(x4)
  x4 = tf.keras.layers.Conv2D(filters=channel//3, kernel_size=(1, 3), padding='same', activation='relu')(x4)
  x4 = tf.keras.layers.BatchNormalization()(x4)
  x4 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(3, 1), padding='same', activation='relu')(x4)
  x4 = tf.keras.layers.BatchNormalization()(x4)
  x4 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(1, 3), padding='same', activation='relu')(x4)
  x4 = tf.keras.layers.BatchNormalization()(x4)

  print(x1.shape,x2.shape,x3.shape,x4.shape)
  combined_x = tf.keras.layers.Concatenate(axis=-1)([x1, x2, x3, x4])

  return tf.keras.layers.Add()([combined_x, x/0.5])

In [25]:
input_x = tf.keras.Input(shape=(224, 224, 3))
x_1 = vgg16.get_layer('input_1')(input_x)

x_2 = vgg16.get_layer('block1_conv1')(x_1)
x_3 = vgg16.get_layer('block1_conv2')(x_2)
x_4 = vgg16.get_layer('block1_pool')(x_3)


x_5 = vgg16.get_layer('block2_conv1')(x_4)
x_6 = vgg16.get_layer('block2_conv2')(x_5)
x_7 = vgg16.get_layer('block2_pool')(x_6)


x_8 = vgg16.get_layer('block3_conv1')(x_7)
x_9 = vgg16.get_layer('block3_conv2')(x_8)
x_10 = vgg16.get_layer('block3_conv3')(x_9)
x_11 = vgg16.get_layer('block3_pool')(x_10)


x_12 = vgg16.get_layer('block4_conv1')(x_11)
x_13 = vgg16.get_layer('block4_conv2')(x_12)
x_14 = vgg16.get_layer('block4_conv3')(x_13)
x_15 = vgg16.get_layer('block4_pool')(x_14)

X1 = vgg16.get_layer('block4_conv3')(rfem(x_13))

x_16 = vgg16.get_layer('block5_conv1')(x_15)
x_17 = vgg16.get_layer('block5_conv2')(x_16)
x_18 = vgg16.get_layer('block5_conv3')(x_17)
x_19 = vgg16.get_layer('block5_pool')(x_18)

X2 = x_18(rfem(X1))

x_20 = tf.keras.layers.Conv2D(filters=1024, kernel_size=3, padding='same', activation='relu', name='block6_conv1')(x_19)
x_21 = tf.keras.layers.MaxPool2D(pool_size=2, name='block6_pool')(x_20)

X3 = x_20(rfem(X2))


x_22 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=2, padding='same', activation='relu', name='block7_conv1')(x_21)
x_23 = tf.keras.layers.MaxPool2D(pool_size=2, name='block7_pool')(x_22)

X4 = x_22(rfem(X3))

x_24 = tf.keras.layers.Flatten()(x_23)
x_25 = tf.keras.layers.Dense(units=4096, activation='relu', name='dense_layer')(x_24)
x_26 = tf.keras.layers.Dense(units=7, activation='softmax', name='final_layer')(x_25)



X41= ftb(X4)
X42= rfem(X41)

X31= ftb(X3, X4)
X32= rfem(X31)

X21= ftb(X2, X3)
X22= rfem(X21)

X11= ftb(X1, X2)
X12= rfem(X11)


(None, 28, 28, 512)
(None, 28, 28, 512)
(None, 28, 28, 128)
(None, 28, 28, 128)
(None, 28, 28, 128) (None, 28, 28, 128) (None, 28, 28, 128) (None, 28, 28, 128)
(None, 28, 28, 512)
(None, 28, 28, 512)
(None, 28, 28, 128)
(None, 28, 28, 128)
(None, 28, 28, 128) (None, 28, 28, 128) (None, 28, 28, 128) (None, 28, 28, 128)


TypeError: ignored

In [ ]:
vgg16_short = tf.keras.models.Model(vgg16.input, vgg16.layers[-5].output)

model_1 = tf.keras.Sequential([
    vgg16_short, 
    tf.keras.layers.Conv2D(filters=1024, kernel_size=3, padding='same', activation='relu', name='block6_conv1'),
    # tf.keras.layers.Conv2D(filters=1024, kernel_size=3, padding='same', activation='relu'),
    # tf.keras.layers.Conv2D(filters=1024, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, name='block6_pool'),
    tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=2, padding='same', activation='relu', name='block7_conv1'),
    # tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu'),
    # tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, name='block7_pool'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=4096, activation='relu', name='dense_layer'),
    tf.keras.layers.Dense(units=7, activation='softmax', name='final_layer')
])
model_1.summary()

In [ ]:
model_2 = tf.keras.models.Model(inputs=vgg16.input, outputs=[vgg16_short.get_layer('block4_conv2').output,
                                                            vgg16_short.get_layer('block4_conv3').output,
                                                            vgg16_short.get_layer('block5_conv3').output,
                                                            model_1.get_layer('block6_conv1').output,
                                                            model_1.get_layer('block7_conv1').output])